In [1]:
import numpy as np
import os
import gc
import pickle
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential, model_from_json

Using Theano backend.


##### Reconstructing saved model

In [7]:
model = model_from_json(open('my_model_architecture.json').read())
model.load_weights('my_model_weights.h5')
model.compile(optimizer='adadelta', loss='binary_crossentropy')

##### Ask user to input some text

In [8]:
raw_input_text = raw_input("Please enter the contect of your email.")

Please enter the contect of your email.hello adfadf


In [9]:
def review_to_words( raw_text ):
    # Function to convert a raw text to a string of words
    # The input is a single string , and 
    # the output is a single string
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_text, "lxml").get_text() 
    #
    # 2. Remove non-letters        
    #letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = review_text.lower().split()                            
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    #stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    #meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( words ))   

##### Pre-processing the text to be inputted to model

In [15]:
input_text_list = []
input_text_list.append(review_to_words(raw_input_text))
transformer = TfidfTransformer()
loaded_vec = CountVectorizer(decode_error="replace",vocabulary=pickle.load(open("bag_of_words_features.pkl", "rb")))
tfidf = transformer.fit_transform(loaded_vec.fit_transform(input_text_list))
tfidf = tfidf.toarray()

##### Predict the inputed text

In [16]:
prediction_results_class = model.predict_classes(np.asarray(tfidf), verbose=1)
prediction_results_prob = model.predict(np.asarray(tfidf), verbose=0)

ValueError: total size of new array must be unchanged
Apply node that caused the error: Reshape{3}(Elemwise{Add}[(0, 0)].0, TensorConstant{[-1 95 70]})
Toposort index: 43
Inputs types: [TensorType(float32, matrix), TensorType(int64, vector)]
Inputs shapes: [(4347, 70), (3,)]
Inputs strides: [(280, 4), (8,)]
Inputs values: ['not shown', array([-1, 95, 70])]
Outputs clients: [[Join(TensorConstant{2}, Reshape{3}.0, Reshape{3}.0, Reshape{3}.0, Reshape{3}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/Users/Armin/anaconda/lib/python2.7/site-packages/keras/models.py", line 146, in add
    output_tensor = layer(self.outputs[0])
  File "/Users/Armin/anaconda/lib/python2.7/site-packages/keras/engine/topology.py", line 485, in __call__
    self.add_inbound_node(inbound_layers, node_indices, tensor_indices)
  File "/Users/Armin/anaconda/lib/python2.7/site-packages/keras/engine/topology.py", line 543, in add_inbound_node
    Node.create_node

In [ ]:
print prediction_results_class
print prediction_results_prob